# Introduction to Machine Learning
## Home Assignment 1

In [6]:

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random

In [7]:
# We get the data 
_URL = 'http://www.di.ens.fr/appstat/spring-2020/project/data.zip'

path_to_zip = tf.keras.utils.get_file('data.zip', origin=_URL, extract=True)

data_path = os.path.join(os.path.dirname(path_to_zip), 'data')

training_data_path = os.path.join(data_path, 'train')

test_data_path = os.path.join(data_path, 'test')

In [8]:
# We define the constants

number_of_letters = 6000
number_of_samples_per_letter = 2000
pixel_height = 28
pixel_width = 28
batch_size = 50
shape = (pixel_height*pixel_width)
seed = 54
# We define a flow of images: 

def generate (dir_path): 
    training_data = []
    training_labels = []
    for item in os.listdir(dir_path):
        tmp = os.path.join(dir_path,item)
        if os.path.isdir(tmp):
            d, l = generate (tmp)
            training_data = training_data + d
            training_labels = training_labels + l
        elif tmp[-3:] == "png":
            im = Image.open(tmp)
            training_data.append(np.reshape(np.asarray(im), shape))
            if(dir_path[-1:] == 'A'):
                training_labels.append(1)
            else: 
                training_labels.append(-1)
    return training_data, training_labels
            
def make (dir_path): 
    d, l = generate(dir_path)
    data = np.array(d)/255
    random.Random(seed).shuffle(data)
    label = np.reshape(np.array(l), (np.shape(l)[0],1))
    random.Random(seed).shuffle(label)
    return data, label



training_data, training_labels = make(training_data_path)
test_data, test_labels = make(test_data_path)

print(np.shape(training_data))
print(np.shape(training_labels))
    


(6000, 784)
(6000, 1)


1. Formalize the problem by defining the input space $X$ , the output space $Y$ and the training data set. What are their dimension?

Having preprocessed the data we are going to use the images as vectors of pixels with size $n = pixel\_width*pixel\_height$. Thus: \
$X$ is $[0, 1]^n$ and $Y$ is $\{-1, 1\}$ and $D = (X \times Y) $



2. a) What are the empirical risk (training error) and the true risk associated with the 0-1 loss? Why is it complicated to minimize the empirical risk in this case

The empirical risk for the 0-1 loss is : $$R_d(f) = \frac{1}{d} \sum_{i = 1}^{d} \mathbb{1}_{f(X_i) \neq Y_i}$$ and the true risk is : $$R(f) = \mathbb{E}\{\mathbb{1}_{f(X) \neq Y} | D \}$$
The empirical risk here is of the 0-1 loss function is non-convex and discontinuous, thus (sub)gradient methods cannot be applied. Any iterative method would be exponential in the input size thus the choice of the 0-1 loss would be inadequate.

2. b) Why should we use the test data to assess the performance ? 

Test data should be from the same statistical distribution as the training data, and should not have been fed to the learning model before (otherwise there is no point given the model already adjusted to that data before)

2. c) Recall the definition of the optimization problems associated with the linear least square regression andthe linear logistic regression.

We start firstly with linear regression: 
$$f_n\in argmin_{f \in \mathbb F}R_n(f) := argmin_{f \in \mathbb F}\frac{1}{n}\sum_{i = 1}^{n}l(x_i, y_i)$$
with $l$ being the loss function associated to the problem. 
In linear regression, we usually choose the square loss, and as the name states we choose a linear model: $f(x) = \theta^T x$. Thus the linear regression problem becomes: 
$$min_{\theta \in \mathbb R^n} R_n(\theta) := min_{\theta \in \mathbb R^n}\frac{1}{n}\sum_{i = 1}^{n}( \theta^T x_i - y_i)^2$$
That can simply be rewritten to: 
$$min_{\theta \in \mathbb R^n} R_n(\theta) := min_{\theta \in \mathbb R^n} \frac{1}{n} || Y - X\theta ||_2^2 $$
with $X \in \mathbb R^{d*n}$ being the design matrix having at the row $i$ $x_i^T$, and $Y \in \mathbb R^d$ being the vector of outputs with $y_i$ at the position $i$

For logistic regression we have: 
$$f_n\in argmin_{f \in \mathbb F}R_n(f) := argmin_{f \in \mathbb F}\frac{1}{n}\sum_{i = 1}^{n}l(x_i, y_i)$$
with $l$ being the loss function associated to the problem. 
In logistic regression, we try to maximize the log-likelihood of an estimator and we end up with a minimization problem using the logistic loss, we choose a linear model: $f(x) = \theta^T x$. Thus the logistic regression problem becomes: 

$$min_{\theta \in \mathbb R^n} R_n(\theta) := min_{\theta \in \mathbb R^n}\frac{1}{n}\sum_{i = 1}^{n}log(1 + e^{−y_i \theta^T x_i}))$$


In [38]:
#we define here some utility functions to compute the regression sum and the gradient

def logistic_regression_loss (theta, x, y): 
    print(np.shape(1/np.shape(y)[0] * np.sum(np.log(1+np.exp(-y*x@theta)), axis=0)))
    return 1/np.shape(y)[0] * np.sum(np.log(1+np.exp(-y*x@theta)), axis=0)

def logistic_gradient (theta, x, y):
    k = -y*x
    print(np.shape(k))
    print(np.shape(theta))
    e = np.exp(k@theta)
    print(np.shape(e))
    u_ = k*e
    print("bro")
    print(np.shape(1/np.shape(y)[0] * np.sum(u_/(1+e), axis=0)))
    return np.reshape(1/np.shape(y)[0] * np.sum(u_/(1+e), axis=0), np.shape(theta)) 


def square_loss (theta, x, y): 
    return 1/np.shape(y)[0] * np.sum((x@theta - y)**2, axis=1)

def square_gradient (theta, x, y):
    return 1/np.shape(y)[0]*2*x*square_loss(theta, x, y)           
    
def sigmoid(t):
    return 1/(1 + np.exp(-t))       

def logistic_classify(theta, data): 
    return 2*(sigmoid(data@theta)>0.5) - 1


In [44]:
#here we define the gradient descent and the stochastic gradient descent with backtracking line search (to get an optimal learning rate)


epsilon = 10**-4
eta = 0.009

def backtracking_line_search (f, x, delta_x, grd_x, alpha= 0.25, beta=0.89):
    t = 1
    while (f(x+t*delta_x) >= f(x) + alptha*t*np.transpose(grd_x)@delta_x):
        t *= beta
    return t

def gradient_descent (f, grd, starting_point, eta = eta, epsilon = epsilon, learning_rate_function = backtracking_line_search):
    theta = starting_point
    prev_theta = starting_point
    r = f(theta, training_data, training_labels)
    print("bro")
    while True:
        prev_theta = theta
        f_theta, delta_theta = r, -grd(theta, training_data, training_labels)
        theta += eta*delta_theta#learning_rate_function(lambda x: regression_sum(x, train_data_gen, f), theta, delta_theta, -delta_theta)*
        r = f(theta, training_data, training_labels)
        print(r - f_theta)
        if (np.absolute(r - f_theta) < epsilon):
            break;
    return theta


def stochastic_gradient_descent (f, grd, starting_point, epsilon=epsilon, learning_rate_function = backtracking_line_search) :
    theta = starting_point
    prev_theta = starting_point
    y = 0
    for x_, y_ in training_data_gen:
        x, y = adjust(x_, y_)
        while True:
            prev_theta = theta
            f_theta, delta_theta = r, -grd(theta, x, y)
            theta += delta_theta#learning_rate_function(lambda t: f(t, x, y), theta, delta_theta, -delta_theta)*
            r = f(theta, x, y)
            print(r - f_theta)
            if (np.absolute(r - f_theta) < epsilon):
                break;
    return theta
    

theta = gradient_descent(logistic_regression_loss, logistic_gradient, np.random.uniform(0, 1, (28*28, 1)))
accuracy = 0
for x, y in zip(test_data, test_labels):
    accuracy += sum(logistic_classify(theta, x) == y) 
    
print(accuracy)

(1,)
bro
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51887116]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51887109]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51887102]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51887095]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51887087]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.5188708]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51887072]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51887064]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51887056]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51887048]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.5188704]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51887031]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51887023]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51887014]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51887005]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-1.51886996]
(6000, 784)
(784,

(1,)
[-0.06601904]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.06194714]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.05832199]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.05507256]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.05214225]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0494832]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.04705113]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0448046]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.04270713]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.04072913]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.03884809]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.03704718]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.03531373]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.03363903]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.03201909]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0304551]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.02895242]
(6000, 7

(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00087101]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00085861]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00084658]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0008349]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00082358]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00081258]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00080191]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00079155]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00078149]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00077171]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00076222]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.000753]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00074405]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00073534]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00072688]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00071865]
(6000, 784)
(784, 1)
(6000,

(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00037298]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00037203]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0003711]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00037017]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00036926]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00036835]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00036746]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00036657]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00036569]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00036482]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00036396]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0003631]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00036226]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00036142]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00036059]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00035977]
(6000, 784)
(784, 1)
(6000

(1,)
[-0.00029083]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00029039]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028995]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0002895]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028906]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028862]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028819]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028775]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028731]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028688]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028644]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028601]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028558]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028514]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028471]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028429]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00028386]
(6000,

...

bro
(784,)
(1,)
[-0.00016366]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00016346]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00016326]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00016306]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00016286]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00016266]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00016247]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00016227]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00016207]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00016187]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00016168]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00016148]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00016129]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00016109]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0001609]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0001607]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.000160

(6000, 1)
bro
(784,)
(1,)
[-0.00014037]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00014023]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00014008]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00013994]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0001398]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00013965]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00013951]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00013937]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00013922]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00013908]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00013894]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0001388]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00013866]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00013851]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00013837]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00013823]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)


(784,)
(1,)
[-0.00012337]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012326]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012316]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012305]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012294]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012284]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012273]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012262]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012252]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012241]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012231]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0001222]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0001221]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012199]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012189]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012178]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00012168]


(784,)
(1,)
[-0.00011065]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00011056]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00011048]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0001104]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00011032]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00011023]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00011015]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00011007]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010999]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010991]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010983]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010974]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010966]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010958]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.0001095]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010942]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010934]


bro
(784,)
(1,)
[-0.00010051]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010044]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010038]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010031]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010024]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010018]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010011]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-0.00010004]
(6000, 784)
(784, 1)
(6000, 1)
bro
(784,)
(1,)
[-9.99747349e-05]
728


In [ ]:
def k_nn (k, stream, distance, cross_validation_percentage):
    return 0